In [99]:
import pandas as pd
import numpy as np
from gamspy import Container, Set, Parameter, Variable, Equation, Model, Sum, Sense

In [100]:
def binomial (n, p, size):
    """
    Generate random numbers following a binomial distribution.

    Parameters:
    - n:    Number of trials (number of times an event is repeated).
    - p:    Probability of success for each trial.
    - size: Number of times to repeat the binomial experiment.

    Returns:
    - An array of random numbers representing the number of successes in each experiment.
    """
    return np.random.binomial(n, p, size)

def randint (low, high, size):
    """
    Generate random integers from a discrete uniform distribution.

    Parameters:
    - low:  The minimum (inclusive) value of the range.
    - high: The maximum (exclusive) value of the range.
    - size: The number of random integers to generate.

    Returns:
    - An array of random integers within the specified range.
    """
    return np.random.randint (low, high, size)

# QUESTION 1
There are n products to be produced $\newline$
There are m suppliers which provide the parts to produce the products $\newline$
A product i needs $a_{ij}$ parts ($a_{ij >= 0}$, $i = 1\rightarrow n$, $j=1\rightarrow m $) $\newline$
The preorder cost for part $j$ is $b_j$ $\newline$ 
The demand for the products is random vector $D = (D_1,..., D_n)$ $\newline$
Cost additionaly $l_i$ to satisfy a unit of demand for product $i$, and unit selling price of that unit is $q_i$ $\newline$
The number of parts ordered is $x_j$, the number of units produced is $z_i$, the numbers of remaining parts unused is $y_j$


### Second stage
The second stage is mentioned first to emphasize the importance of understanding the uncertainty in case of solving a stochastic problem. This approach aligns with the principles of stochastic programming, where the second-stage decisions are made after observing the realization of random data and are contingent on the first-stage decisions. $\newline$
In this stage, we want to minimize the cost of production. Specifically, we need to find the optimal cost of filling an additional portion of demand while reselling the salvage parts to minimize the loss of not using the parts. $\newline$
$$ 
min_{x,y} \text { } Z=\sum^n_{i=1} (l_i-q_i)z_i - \sum^m_{j=1}s_jy_j \newline \text {subject to} \newline
y_j=x_j- \sum_{i=1}^n a_{ij}z_i, j=1,...,m \newline
0 \leq z_i \leq d_i, i=1,...,n \newline
y_j \geq 0, j=1,...,m
$$

### First stage
In this stage, $x$ needs to be decided before a realization of the demand $D$, which makes it a $\textit {here and now}$ decisions $\newline$
Moreover, the expected value of $Z$ should also be taken into account. To be specific, that is the optimal value $Q(x)$ of the second stage ($Q(x) = E[Z(z,y)]$) $\newline$
Therefore, we yield this problem below: 
$$min \text { } g(x,y,z) = b^Tx + Q(x) = b^Tx + E[Z(z,y)]$$

### Big problem
We need to solve this
$$
min \text { } b^Tx + \sum^S_{i=1} p_s[(l_i-q_i)^Tz_i - s^T_iy_i] \newline
\text {subject to} \newline
y \geq 0 \newline
0 \leq z \leq d \newline
x \geq 0 \newline
$$

In [101]:
n   = 8           # number of products
m   = 5           # number of parts to be ordered
S   = 2           # number of scenarios
p_s = [0.5, 0.5]  # probability of each scenario / density

D = np.array([binomial (10, 0.5, n),
              binomial (10, 0.5, n)]) # demand for each product in each scenario matrix [n x S]

b = randint(5, 10, m) # preorder cost for each part

A = []
for i in range(n):
    A.append(np.array(randint(1,10,m)))

A = np.array(A) # bill of materials matrix [m x n]
# A = np.transpose(A)

s = randint (1, 10, m)       # salvage values
l = randint (100, 200, n)    # additional costs
q = randint (1000, 1400, n)  # unit selling prices

c = l - q #cost coefficients

D

array([[7, 6, 6, 4, 5, 6, 5, 3],
       [4, 5, 6, 6, 5, 6, 5, 2]])

In [102]:
print('Preorder cost: ', b)
print('Material usage:\n', A)
print('Salvage value: ', s)
print('Additional producing cost: ', l)
print('Selling prices: ', q)
print('Cost coefficient:', c)
# print('Demand:\n', D)

Preorder cost:  [8 8 5 6 7]
Material usage:
 [[4 3 7 2 8]
 [2 7 7 1 7]
 [4 6 8 5 7]
 [4 8 8 4 3]
 [6 7 6 5 8]
 [2 9 1 5 5]
 [6 8 2 6 4]
 [8 6 4 5 1]]
Salvage value:  [8 5 9 9 3]
Additional producing cost:  [185 197 140 180 138 135 184 143]
Selling prices:  [1044 1153 1135 1389 1227 1384 1087 1207]
Cost coefficient: [ -859  -956  -995 -1209 -1089 -1249  -903 -1064]


In [103]:
# df_parts_usage = []
# for i in range(m):
#     df_parts_usage.append([PARTS_NAME[i]])
#     for j in range(n):
#         df_parts_usage[i].append(A[i][j])

# df_parts_usage = pd.DataFrame(
#     data = df_parts_usage,
#     columns=[''] + PRODUCTS_NAME
# ).set_index('')

# df_parts_usage

In [104]:
m = Container()


In [105]:
products = ['product1', 'product2', 'product3', 'product4', 'product5','product6','product7','product8']
i = Set(
    container=m,
    name='products',
    description='products of the firm',
    records=products
)
i.records

,uni,element_text
0,product1,
1,product2,
2,product3,
3,product4,
4,product5,
5,product6,
6,product7,
7,product8,


In [106]:
parts = ['part1', 'part2', 'part3', 'part4', 'part5']
j = Set(
    container=m,
    name='parts',
    description='Parts to build the products',
    records=parts
)
j.records

,uni,element_text
0,part1,
1,part2,
2,part3,
3,part4,
4,part5,


In [107]:
B = Parameter(
    container=m,
    name='Preorder_price',
    domain=j,
    description='Price to preorder a part',
    records=b
)
B.records

,parts,value
0,part1,8.0
1,part2,8.0
2,part3,5.0
3,part4,6.0
4,part5,7.0


In [108]:
a = Parameter (
    container=m,
    name='parts_usages',
    domain=[i, j],
    description='Numbers of parts needed for n products',
    records= A
)
a.records

,products,parts,value
0,product1,part1,4.0
1,product1,part2,3.0
2,product1,part3,7.0
3,product1,part4,2.0
4,product1,part5,8.0
5,product2,part1,2.0
6,product2,part2,7.0
7,product2,part3,7.0
8,product2,part4,1.0
9,product2,part5,7.0


In [109]:
sc = Set(
    container=m,
    name='Scenario',
    description="Two scenario of the demand",
    records = ['Scenario 1', 'Scenario 2']
)

d = Parameter(
    container=m,
    name='Demand',
    domain=[sc,i],
    description="Demand for products",
    records = D
)

d.records

,Scenario,products,value
0,Scenario 1,product1,7.0
1,Scenario 1,product2,6.0
2,Scenario 1,product3,6.0
3,Scenario 1,product4,4.0
4,Scenario 1,product5,5.0
5,Scenario 1,product6,6.0
6,Scenario 1,product7,5.0
7,Scenario 1,product8,3.0
8,Scenario 2,product1,4.0
9,Scenario 2,product2,5.0


In [110]:
sal = Parameter(
    container=m,
    name='Salvage_price',
    domain=j,
    records=s
)
sal.records

,parts,value
0,part1,8.0
1,part2,5.0
2,part3,9.0
3,part4,9.0
4,part5,3.0


In [111]:
Apc = Parameter(
    container=m,
    name='Addition_producing_cost',
    domain=i,
    records = l
)
Apc.records

,products,value
0,product1,185.0
1,product2,197.0
2,product3,140.0
3,product4,180.0
4,product5,138.0
5,product6,135.0
6,product7,184.0
7,product8,143.0


In [112]:
Sp = Parameter(
    container=m,
    name='Selling_price',
    domain=i,
    records=q
)
Sp.records

,products,value
0,product1,1044.0
1,product2,1153.0
2,product3,1135.0
3,product4,1389.0
4,product5,1227.0
5,product6,1384.0
6,product7,1087.0
7,product8,1207.0


In [113]:
x = Variable(
    container=m,
    name='x',
    domain=j,
    description='Number of preordered parts'
)

z = Variable(
    container=m,
    name='z',
    domain=i,
    description='Number of produced products'
)

y = Variable(
    container=m,
    name='y',
    domain=j,
    description='Number of unused parts'
)

In [114]:
con_1_1 = Equation(
    container=m,
    name='Constraint_1_1',
    domain=i,
    definition= x>=0
)

con_1_2 = Equation(
    container=m,
    name='Constraint_1_2',
    domain=j,
    definition=y>=0
)

con_1_3 = Equation(
    container=m,
    name='Constraint_1_3',
    domain=i,
    definition=z>=0
)

con_1_4 = Equation(
    container=m,
    name='Constraint_1_4',
    domain=i
    definition= y == x-A
)
con_1_4[i] = x - Sum(j, a[i,j]*z[i])

con_1_5 = Equation(
    container=m,
    name='Constraint_1_5',
    domain=i,
    definition= z <= d
)

$$
min \text { } b^Tx + \sum^n_{i=1} p_s[(l_i-q_i)^Tz_i - s^T_jy_j]$$

In [118]:

obj = b*x + sum()

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
production = Model(
    container=m,
    name='production',
    equations=m.getEquations(),
    problem='LP',
    sense=Sense.MIN,
    objective=obj
)

In [ ]:
import sys
production.solve(output=sys.stdout)

--- Job _job_756a863f-0277-41ac-965b-cd52ef0d4a53.gms Start 12/03/23 13:40:15 45.4.0 19dc3313 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\Users\DUC_AN\.conda\envs\gamspy\Lib\site-packages\gamspy_base\gmsprmNT.txt
--- GAMS Parameters defined
    LP CPLEX
    MIP CPLEX
    RMIP CPLEX
    NLP CONOPT
    MCP PATH
    MPEC NLPEC
    RMPEC CONVERT
    CNS CONOPT
    DNLP CONOPT
    RMINLP CONOPT
    MINLP SBB
    QCP CONOPT
    MIQCP SBB
    RMIQCP CONOPT
    EMP CONVERT
    Input C:\Users\DUC_AN\AppData\Local\Temp\tmpmdbtkhq3\_job_756a863f-0277-41ac-965b-cd52ef0d4a53.gms
    Output C:\Users\DUC_AN\AppData\Local\Temp\tmpmdbtkhq3\_job_756a863f-0277-41ac-965b-cd52ef0d4a53.lst
    Save C:\Users\DUC_AN\AppData\Local\Temp\tmpmdbtkhq3\_save_c1dbcfc7-24b8-44d5-af47-fc611ebe2c46.g00
    ScrDir C:\Users\DUC_AN\AppData\Local\Temp\tmpmdbtkhq3\225a\
    SysDir C:\Users\DUC_AN\.conda\envs\gamspy\Lib\site-packages\gamspy_base\
    CurDir C:\Users\DUC_AN\AppData\Local\Temp\tmpmdbtkhq3\
    LogOption 

GamsExceptionExecution: 

================================================================================
Error Summary
================================================================================
  30  Constraint_1_1 .. x =g= 0;
****                  $148  $148
**** 148  Dimension different - The symbol is referenced with more/less
****         indices as declared

GAMS return code not 0 (2), check C:\Users\DUC_AN\AppData\Local\Temp\tmpmdbtkhq3\_job_756a863f-0277-41ac-965b-cd52ef0d4a53.lst for more details